# Rechtspraak, LI & citations to DynamoDB ETL

In [ ]:
!pip install boto3

In [1]:
import boto3
from boto3.dynamodb import conditions
from boto3.dynamodb.conditions import Key, Attr
from decimal import Decimal
import json
import csv
from pprint import pprint
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import string
from datetime import datetime
import dateutil.parser

- rs = rechtspraak data
- li = legal intelligence data
---
## Raw Datasets

In [2]:
#pd.set_option('display.max_columns', None)
df_rs_cases_raw = pd.read_csv("../data/case.csv", dtype=str).replace({np.nan: None})  #, nrows=500000).replace(np.nan, None, regex=True)
#df_rs_cases_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case.csv", dtype=str).replace({np.nan: None})  #, nrows=500000).replace(np.nan, None, regex=True)
df_rs_cases_raw.head()

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,...,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:GHAMS:1913:15,1913-03-31,Onbekend,None,NL,None,None,None,None,NL,...,None,None,None,None,None,None,None,None,None,None
1,ECLI:NL:GHAMS:1913:28,1913-02-28,Onbekend389,None,NL,None,None,None,None,NL,...,None,None,None,None,None,None,None,None,None,None
2,ECLI:NL:GHAMS:1913:19,1913-03-17,Onbekend,None,NL,None,None,None,None,NL,...,None,None,None,None,None,None,None,None,None,None
3,ECLI:NL:GHAMS:1913:4,1913-02-28,Onbekend,None,NL,None,None,None,None,NL,...,None,None,None,None,None,None,None,None,None,None
4,ECLI:NL:GHAMS:1913:20,1913-03-10,Onbekend,None,NL,None,None,None,None,NL,...,None,None,None,None,None,None,None,None,None,None


In [6]:
df_rs_opinions_raw = pd.read_csv("../data/case_opinion_from_advocate_general.csv", dtype='str').replace({np.nan: None})
#df_rs_opinions_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/case_opinion_from_advocate_general.csv", dtype='str').replace({np.nan: None})
df_rs_opinions_raw.head()

,case_id,date,case_number,description,language,venue,abstract,procedure_type,lodge_date,country,subject,authority,legal_references,related_cases,alternative_sources,full_textcase_id,date.1,case_number.1,description.1,language.1,venue.1,abstract.1,procedure_type.1,lodge_date.1,country.1,subject.1,authority.1,legal_references.1,related_cases.1,alternative_sources.1,full_text
0,ECLI:NL:PHR:1913:AG1763,1913-04-18,x47,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1763\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ECLI:NL:PHR:1913:AG1764,1913-04-18,x48,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1764\n,"\n \n <rdf:list xmlns:rdf=""htt...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ECLI:NL:PHR:1913:BG9429,1913-04-21,None,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:BG9429\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ECLI:NL:PHR:1913:AG1832,1913-03-07,X46,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1832\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ECLI:NL:PHR:1913:AG1762,1913-06-20,x49,None,NL,None,None,None,None,NL,None,Parket bij de Hoge Raad,None,Arrest Hoge Raad: ECLI:NL:HR:1913:AG1762\n,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [2]:
df_li_cases_raw = pd.read_csv("../data/legal_intelligence_cases.csv", dtype=str).replace({np.nan: None}).drop_duplicates(subset ="ecli")
#df_li_cases_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legal_intelligence_cases.csv", dtype=str).replace({np.nan: None}).drop_duplicates(subset ="ecli")
df_li_cases_raw.head()

,Unnamed: 0,IssueNumber,Id,Title,DisplayTitle,DisplaySubtitle,abstract,Url,OriginalUrl,Jurisdiction,...,authority,case_number,date,lodge_date,DateAdded,Sources,UrlWithAutoLogOnToken,LawArea,ecli,PublicationNumber
0,0,None,4968258,Casus Bij besluit van 12 april 1994 verlenen g...,Casus Bij besluit van 12 april 1994 verlenen g...,"Doorduijn, L.F. - Milieurecht Totaal - Module-...",Casus Bij besluit van 12 april 1994 verlenen g...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da785f3d8d...,Nederland,...,Raad van State,E03951413,19960101,19960916,20110520,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_AP7727,None
1,1,None,4968759,M.N. Boeve Casus GS van Noord-Brabant hebben (...,M.N. Boeve Casus GS van Noord-Brabant hebben (...,"Boeve, M.N. - Milieurecht Totaal - Module-seri...",M.N. Boeve Casus GS van Noord-Brabant hebben (...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dae9667789...,Nederland,...,Raad van State,E01.94.0417 ECLI:NL:RVS:1996:AP7883,19960101,19960708,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_AP7883,None
2,2,None,4968382,Naast woning gelegen huisje vormt ┌┌n geheel m...,Naast woning gelegen huisje vormt ┌┌n geheel m...,Redactie - Module-serie - Module Afval - Milie...,Naast woning gelegen huisje vormt ┌┌n geheel m...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da7b16fb16...,Nederland,...,Gerechtshof Amsterdam,95/4927-E,19960101,19961203,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_GHAMS_1996_AS4281,None
3,3,None,4968678,Geen beroep mogelijk tegen weigering bestemmin...,Geen beroep mogelijk tegen weigering bestemmin...,Redactie - Module-serie - Module Ruimtelijke O...,De raad van de gemeente Waspik wees een verzoe...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/dad97422ad...,Nederland,...,Raad van State,H01950354,19960101,19960201,20110628,Module Ruimtelijke Ordening,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_RVS_1996_ZF1978,None
4,4,None,4968777,Bodemverontreiniging; saneringskosten komen oo...,Bodemverontreiniging; saneringskosten komen oo...,"Kruimel, J.P. - Milieurecht Totaal",Bodemverontreiniging; saneringskosten komen oo...,http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/daf99e49b6...,Nederland,...,Gerechtshof Den Haag,95/1803,19960101,19961016,20110628,Milieurecht Totaal,http://www.legalintelligence.com/documents/496...,['Ruimtelijk Bestuursrecht/Milieurecht/Energie...,ECLI_NL_GHSGR_1996_AS3490,None


In [7]:
df_li_cases_raw['Id'].describe()

count         661
unique        661
top       5195871
freq            1
Name: Id, dtype: object

In [2]:
df_rs_case_citations_raw = pd.read_csv("../data/caselaw_citations.csv").replace({np.nan: None})
#df_rs_case_citations_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/caselaw_citations.csv").replace({np.nan: None})
df_rs_case_citations_raw.head()

,source_ecli,source_paragraph,target_ecli,target_paragraph
0,ECLI:NL:HR:1913:13,None,ECLI:NL:HR:1913:129,None
1,ECLI:NL:HR:1913:130,None,ECLI:NL:HR:1913:129,None
2,ECLI:NL:HR:1913:131,None,ECLI:NL:HR:1913:129,None
3,ECLI:NL:HR:1913:132,None,ECLI:NL:HR:1913:129,None
4,ECLI:NL:HR:1913:133,None,ECLI:NL:HR:1913:129,None


In [3]:
df_rs_legislation_citations_raw = pd.read_csv("../data/legislation_citations.csv").replace({np.nan: None})
#df_rs_legislation_citations_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/legislation_citations.csv").replace({np.nan: None})
df_rs_legislation_citations_raw.head()

,source_ecli,source_paragraph,target_article,target_article_paragraph,target_article_webpage
0,ECLI:NL:GHAMS:1962:AX8440,None,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,None,http://wetten.overheid.nl/id/BWBR0005537/2016-...
1,ECLI:NL:GHAMS:1962:AX8440,None,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,None,http://wetten.overheid.nl/id/BWBR0005537/2009-...
2,ECLI:NL:GHAMS:1962:AX8440,None,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,None,http://wetten.overheid.nl/id/BWBR0005537/2018-...
3,ECLI:NL:GHAMS:1962:AX8441,None,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,None,http://wetten.overheid.nl/id/BWBR0005537/2016-...
4,ECLI:NL:GHAMS:1962:AX8441,None,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,None,http://wetten.overheid.nl/id/BWBR0005537/2009-...


In [8]:
df_country_raw = pd.read_csv("../data/countries.csv").replace({np.nan: None})
#df_country_raw = pd.read_csv("https://maastrichtuniversity-ids-open.s3.eu-central-1.amazonaws.com/rechtspraak/countries.csv").replace({np.nan: None})
df_country_raw.head()

,country_id,eu,name,flag
0,AT,1,Austria,https://www.countryflags.io/at/flat/64.png
1,BE,1,Belgium,https://www.countryflags.io/be/flat/64.png
2,BG,1,Bulgaria,https://www.countryflags.io/bg/flat/64.png
3,CH,0,Switzerland,https://www.countryflags.io/ch/flat/64.png
4,CZ,1,Czechia,https://www.countryflags.io/cz/flat/64.png


#### stats raw data

In [9]:
print('number of rs-cases: ', len(df_rs_cases_raw))
print('number of opinions: ', len(df_rs_opinions_raw))
print('number of li-cases: ', len(df_li_cases_raw))
print('number of legislation citations: ', len(df_rs_legislation_citations_raw))
print('number of case citations: ', len(df_rs_case_citations_raw))
print('number of countries: ', len(df_country_raw))

number of rs-cases:  2632420
number of opinions:  65692
number of li-cases:  661
number of legislation citations:  18947
number of case citations:  758268
number of countries:  36


---
## Utils

In [3]:
def clean_strings(column):
    cleaned = []
    for s in column:
        if s is not None:
            cleaned.append("".join(filter(lambda char: char in string.printable, s)))
        else:
            cleaned.append("")
    return [i[0:250] for i in cleaned]

---
## Transformation

Unify eclis:

In [5]:
for i, li_case in df_li_cases_raw.iterrows():
    ecli = li_case.ecli
    new_ecli = ecli.replace('_', ':')
    df_li_cases_raw.at[i, 'ecli'] = new_ecli

#### RS cases

In [4]:
map_rs_cases = {'case_id':'ecli',  # id = ecli or country/court id
          'date':'date_judgement',
          'subject':'domain',
          'authority':'instance',
          'case_number':'case_number',
          'description':'inhoudsindicatie',
          'language':'language',
          'country':'jurisdiction_country',
          'venue':'jurisdiction_city',
          'abstract':'uitspraak_info',
          'procedure_type':'procedure_type',
          'alternative_sources':'alternative_sources',
          'legal_references':'legislation_title'}
df_rs_cases = df_rs_cases_raw[map_rs_cases.keys()].rename(columns=map_rs_cases)
df_rs_cases['doctype'] = 'decision'
df_rs_cases['data_source'] = 'Rechtspraak'

# clean strings and transform date type
#df_rs_cases['uitspraakdatum'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_rs_cases['uitspraakdatum']]
df_rs_cases['inhoudsindicatie'] = clean_strings(df_rs_cases['inhoudsindicatie'])
df_rs_cases['alternative_sources'] = clean_strings(df_rs_cases['alternative_sources'])
df_rs_cases['domain'] = df_rs_cases['domain'].apply(lambda x: set(x.split("; ")) if x is not None else x)

KeyboardInterrupt: 

#### RS opinions

In [11]:
map_rs_opinions = {'case_id':'ecli',  # id = ecli or country/court id
          'date':'uitspraakdatum',
          'subject':'domain',
          'authority':'instance',
          'case_number':'case_number',
          'description':'inhoudsindicatie',
          'language':'language',
          'country':'jurisdiction_country',
          'venue':'jurisdiction_city',
          'abstract':'conclusie_info',
          'procedure_type':'procedure_type',
          'lodge_date':'lodge_date',
          'alternative_sources':'link',
          'legal_references':'legislation_title'}


df_rs_opinions = df_rs_opinions_raw[map_rs_opinions.keys()].rename(columns=map_rs_opinions)
df_rs_opinions['doctype'] = 'opinion'

#df_rs_opinions['uitspraakdatum'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_rs_opinions['uitspraakdatum']]
df_rs_opinions['conclusie_info'] = clean_strings(df_rs_opinions['conclusie_info'])
df_rs_opinions['domain'] = df_rs_opinions['domain'].apply(lambda x: set(x.split("; ")) if x is not None else x)

#### LI cases

In [17]:
map_li = {
    'ecli':'ecli',
    'Title':'title_li',
    'date':'publication_date_li',
    'abstract':'summary_li',
    'LawArea':'domain_li',
    'authority':'instance_li',
    'Url':'Url_li',
    'DisplayTitle':'DisplayTitle_li',
    'OriginalUrl':'OriginalUrl_li',
    'Jurisdiction':'jurisdiction_country_li',
    'DocumentType':'DocumentType_li',
    'case_number':'case_number_li',
    'PublicationNumber':'PublicationNumber_li',
    'IssueNumber':'IssueNumber_li',
    'Id':'id_li',
    'lodge_date':'date_judgement_li',
    'DateAdded':'date_published_online_li',
    'Sources':'Sources_li',
    'UrlWithAutoLogOnToken':'UrlWithAutoLogOnToken_li',
    'DisplaySubtitle':'DisplaySubtitle_li'
}
df_li_cases = df_li_cases_raw[map_li.keys()].rename(columns=map_li)
df_li_cases['doctype'] = 'decision'

# clean strings and transform date type
#df_li_cases['publication_date_li'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_li_cases_raw['date']]
df_li_cases['publication_date_li'] = df_li_cases['publication_date_li'].apply(lambda i: dateutil.parser.parse(i).date().strftime('%Y-%m-%d') if i is not None else None)
#df_li_cases['lodge_date_li'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_li_cases_raw['lodge_date']]
df_li_cases['date_judgement_li'] = df_li_cases['date_judgement_li'].apply(lambda i: dateutil.parser.parse(i).date().strftime('%Y-%m-%d') if i is not None else None)
#df_li_cases['DateAdded_li'] = [dateutil.parser.parse(i).isoformat() if i is not None else '1900-01-01T00:00:00' for i in df_li_cases_raw['DateAdded']]
df_li_cases['date_published_online_li'] = df_li_cases['date_published_online_li'].apply(lambda i: dateutil.parser.parse(i).date().strftime('%Y-%m-%d') if i is not None else None)
df_li_cases['title_li'] = clean_strings(df_li_cases['title_li'])
df_li_cases['summary_li'] = clean_strings(df_li_cases['summary_li'])
df_li_cases['DisplayTitle_li'] = clean_strings(df_li_cases['DisplayTitle_li'])
df_li_cases['instance_li'] = clean_strings(df_li_cases['instance_li'])
df_li_cases['DisplaySubtitle_li'] = clean_strings(df_li_cases['DisplaySubtitle_li'])
df_li_cases['domain_li'] = df_li_cases['domain_li'].apply(lambda x: set(x.strip("']['").split("', '")))

#### RS case citations

In [13]:
map_case_citations = {
    'source_ecli':'id',
    'target_ecli':'target_ecli',
}
df_rs_case_citations = df_rs_case_citations_raw[map_case_citations.keys()].rename(columns=map_case_citations)
df_rs_case_citations['doctype'] = 'c-citation-' + df_rs_case_citations['target_ecli'].map(str)

#### RS legislation citations

In [14]:
map_legislation_citations = {
    'source_ecli':'id',
    'target_article':'target_legislation_url',
    'target_article_webpage':'target_legislation_article_url'
}

df_rs_legislation_citations = df_rs_legislation_citations_raw[map_legislation_citations.keys()].rename(columns=map_legislation_citations)
df_rs_legislation_citations['doctype'] = 'l-citation-' + df_rs_legislation_citations['target_legislation_article_url'].map(str)

#### RS case related decisions

In [6]:
map_case_related_decisions = {
    'case_id':'id',
    'related_cases':'predecessor_successor_eclis'
    }
df_rs_related_cases = df_rs_cases_raw.dropna(subset=['related_cases'])[['case_id', 'related_cases']]
df_rs_related_cases = df_rs_related_cases[map_case_related_decisions.keys()].rename(columns=map_case_related_decisions)
df_rs_related_cases['predecessor_successor_eclis'] = df_rs_related_cases['predecessor_successor_eclis'].apply(lambda x: x.strip())
df_rs_related_cases['doctype'] = 'related-' + df_rs_related_cases['predecessor_successor_eclis'].map(str)
df_rs_related_cases.head()

,id,predecessor_successor_eclis,doctype
125,ECLI:NL:HR:1913:AG1763,Conclusie: ECLI:NL:PHR:1913:AG1763,related-Conclusie: ECLI:NL:PHR:1913:AG1763
129,ECLI:NL:HR:1913:AG1764,Conclusie: ECLI:NL:PHR:1913:AG1764,related-Conclusie: ECLI:NL:PHR:1913:AG1764
139,ECLI:NL:HR:1913:BG9429,Conclusie: ECLI:NL:PHR:1913:BG9429,related-Conclusie: ECLI:NL:PHR:1913:BG9429
293,ECLI:NL:HR:1913:AG1832,Conclusie: ECLI:NL:PHR:1913:AG1832,related-Conclusie: ECLI:NL:PHR:1913:AG1832
440,ECLI:NL:HR:1913:AG1762,Conclusie: ECLI:NL:PHR:1913:AG1762,related-Conclusie: ECLI:NL:PHR:1913:AG1762


#### Courts

In [20]:
df_court = pd.DataFrame()
courts_list = list(set(df_rs_cases_raw.authority.unique().tolist() + df_rs_opinions_raw.authority.unique().tolist() + df_li_cases_raw.authority.unique().tolist()))
courts_list = [i.replace('"','-') for i in courts_list]
courts_list.extend(['Other'])

#df_court['name'] = clean_strings(courts_list)
df_court['id'] = clean_strings(courts_list)  # court name
df_court.loc[:,'type'] = None
df_court.loc[:,'level'] = None
df_court.loc[:,'country'] = None
df_court.loc[:,'language'] = None
df_court.loc[:,'jurisdiction'] = None
df_court.loc[:,'domain'] = None
df_court.loc[:,'authority_level'] = None
df_court.loc[:, 'doctype'] = 'court'


#### Countries

In [21]:
df_country = df_country_raw.copy()
df_country.loc[:,'language'] = None
df_country.loc[:,'eea'] = 0
df_country = df_country[['country_id','name','language','flag','eu','eea']].rename(columns={'country_id':'id'})
df_country.loc[:, 'doctype'] = 'country'

#### Split data into RS only cases, LI only cases and RS+LI cases:

In [18]:
''' intersection of cases in li and case_raw '''
rsli_intersection = list(set(df_li_cases['ecli']).intersection(set(df_rs_cases['ecli'])))
rs_only = list(set(df_rs_cases['ecli']) - set(rsli_intersection))
li_only = list(set(df_li_cases['ecli']) - set(rsli_intersection))
print('number of li only cases: ', len(li_only))
print('number of rechtspraak only cases: ', len(rs_only))
print('number of overlapping cases: ', len(rsli_intersection))
print('number of rechtspraak total cases: ', len(df_rs_cases_raw))
print('number of li total cases: ', len(df_li_cases_raw))

number of li only cases:  44
number of rechtspraak only cases:  2631803
number of overlapping cases:  617
number of rechtspraak total cases:  2632420
number of li total cases:  661


In [19]:
### build separate dfs for li only and raw only cases
df_rs_only_cases = df_rs_cases[df_rs_cases['ecli'].isin(rs_only)]
df_li_only_cases = df_li_cases[df_li_cases['ecli'].isin(li_only)]


In [20]:
### merge overlapping li and raw cases into one df
df_rsli_intersection_cases = df_li_cases[df_li_cases['ecli'].isin(rsli_intersection)]
df_rsli_intersection_cases.set_index('ecli', inplace=True)
df_rs_cases.set_index('ecli', inplace=True)
df_rsli_intersection_cases = df_rsli_intersection_cases.join(df_rs_cases.drop('doctype', axis=1))
df_rs_cases.reset_index(inplace=True)
df_rsli_intersection_cases.reset_index(inplace=True)

In [21]:
df_rsli_intersection_cases.columns

Index(['ecli', 'title_li', 'publication_date_li', 'summary_li', 'domain_li',
       'instance_li', 'Url_li', 'DisplayTitle_li', 'OriginalUrl_li',
       'jurisdiction_country_li', 'DocumentType_li', 'case_number_li',
       'PublicationNumber_li', 'IssueNumber_li', 'id_li', 'date_judgement_li',
       'date_published_online_li', 'Sources_li', 'UrlWithAutoLogOnToken_li',
       'DisplaySubtitle_li', 'doctype', 'date_judgement', 'domain', 'instance',
       'case_number', 'inhoudsindicatie', 'language', 'jurisdiction_country',
       'jurisdiction_city', 'uitspraak_info', 'procedure_type',
       'alternative_sources', 'legislation_title', 'data_source'],
      dtype='object')

In [24]:
mapped_cases = df_rsli_intersection_cases[['ecli',
                            'date_judgement', 'publication_date_li', 'date_judgement_li', 'date_published_online_li',
                            'case_number', 'case_number_li',
                            'inhoudsindicatie', 'uitspraak_info', 'summary_li',
                            'language',
                            'jurisdiction_country', 'jurisdiction_country_li', 'jurisdiction_city',
                            'domain', 'domain_li',
                            'instance', 'instance_li',
                            'title_li', 'DisplayTitle_li', 'DisplaySubtitle_li',
                            'alternative_sources', 'Url_li', 'OriginalUrl_li', 'UrlWithAutoLogOnToken_li',
                            'procedure_type', 'Sources_li',
                            'PublicationNumber_li', 'IssueNumber_li', 'id_li']]
mapped_cases.to_csv('../data/RS-LI_mapped_decisions.csv', index=False)
#mapped_cases = pd.read_csv('../data/RS-LI_mapped_decisions.csv')

In [20]:
mapped_cases.loc[~(mapped_cases['uitspraakdatum'] == mapped_cases['lodge_date_li'])]

,ecli,uitspraakdatum,publication_date_li,lodge_date,lodge_date_li,DateAdded_li,case_number,case_number_li,inhoudsindicatie,uitspraak_info,summary_li,language,procedure_type,jurisdiction_country,jurisdiction_country_li,jurisdiction_city,domain,domain_li,instance,instance_li,title,title_li,DisplayTitle_li,DisplaySubtitle_li,link,link_li,OriginalUrl_li,UrlWithAutoLogOnToken_li,PublicationNumber_li,IssueNumber_li
242,ECLI:NL:CRVB:1997:ZB7203,1997-10-20,1997-01-01,None,1997-10-23,2011-06-23,96/7800 Algem,96/7800ALGEM ECLI:NL:CRVB:1997:ZB7203,,None,Belanghebbende is jaren lang als docent in die...,NL,Hoger beroep,NL,Nederland,None,{Bestuursrecht},"{Arbeids/Sociaal Recht, Belastingrecht}",Centrale Raad van Beroep,Centrale Raad van Beroep,None,PREMIEHEFFING Geen verzekeringsplicht docent,"V-N 1997/4404, 20 - Met noot - Redactie - Cent...",PREMIEHEFFING Geen verzekeringsplicht docent,"\n \n <rdf:list xmlns:rdf=""htt...",http://www.legalintelligence.com/documents/519...,https://www.navigator.nl/#/document/inod20135f...,http://www.legalintelligence.com/documents/519...,"V-N 1997/4404, 20",VN/1997/000053
243,ECLI:NL:CRVB:1997:ZB7202,1997-10-20,1997-01-01,None,1997-10-23,2011-06-23,96/6702 Algem,96/6702 ECLI:NL:CRVB:1997:ZB7202,,None,Stichting R verzorgt allerlei soorten cursusse...,NL,Hoger beroep,NL,Nederland,None,{Bestuursrecht},"{Arbeids/Sociaal Recht, Belastingrecht, Onbekend}",Centrale Raad van Beroep,Centrale Raad van Beroep,None,PREMIEHEFFING Gastdocenten werkzaam in dienstb...,"V-N 1997/4407, 21 - Met noot - Redactie - Cent...",PREMIEHEFFING Gastdocenten werkzaam in dienstb...,"\n \n <rdf:list xmlns:rdf=""htt...",http://www.legalintelligence.com/documents/519...,https://www.navigator.nl/#/document/inod64ed8f...,http://www.legalintelligence.com/documents/519...,"V-N 1997/4407, 21",VN/1997/000053
344,ECLI:NL:RBGRO:1997:AL0841,1997-11-28,1998-01-01,None,1999-12-17,2010-07-22,18259/HA ZA 96-185,18259/HAZA96-185 ECLI:NL:RBGRO:1997:AL0841,,None,De gemeente heeft primair gesteld dat R. in ha...,NL,None,NL,Nederland,None,None,"{Arbeids/Sociaal Recht, Onbekend, Burgerlijke ...",Rechtbank Groningen,Rechtbank Groningen,None,Vergoeding van schade bestaande uit kosten van...,"AB 1998, 226 - Met noot - Buuren van, P.J.J. -...",Vergoeding van schade bestaande uit kosten van...,"\n \n <rdf:list xmlns:rdf=""htt...",http://www.legalintelligence.com/documents/373...,https://www.navigator.nl/#/document/id19519971...,http://www.legalintelligence.com/documents/373...,"AB 1998, 226",AB/1998/000026
345,ECLI:NL:HR:1998:AA2539,1998-08-26,1998-01-01,None,1999-08-26,2011-06-23,33568,33 568,"\n -<para xmlns=""http://www.rechtspraak.n...",None,"De vof X, die een tuinbouwbedrijf uitoefent en...",NL,Cassatie,NL,Nederland,None,"{Belastingrecht, Bestuursrecht}","{Belastingrecht, Onbekend}",Hoge Raad,Hoge Raad,None,OMZETBELASTING Vergoeding. Door tuinder zelf v...,V-N 1998/47.17 - Met noot - Redactie - Hoge Ra...,OMZETBELASTING Vergoeding. Door tuinder zelf v...,"\n \n <rdf:list xmlns:rdf=""htt...",http://www.legalintelligence.com/documents/519...,https://www.navigator.nl/#/document/inodcdd076...,http://www.legalintelligence.com/documents/519...,V-N 1998/47.17,VN/1998/000047
489,ECLI:NL:RVS:1999:AL2870,1999-11-11,1999-01-01,None,1999-06-28,2011-05-20,E03.96.0413,F03990091 ECLI:NL:RVS:1999:AL2870,,None,HFMW van Rijswick Casus Een Wvo-vergunning is ...,NL,None,NL,Nederland,None,None,{Ruimtelijk Bestuursrecht/Milieurecht/Energier...,Raad van State,Raad van State,None,HFMW van Rijswick Casus Een Wvo-vergunning is ...,HFMW van Rijswick Casus Een Wvo-vergunning is ...,Module-serie - Module Afval - Module Water - M...,"\n \n <rdf:list xmlns:rdf=""htt...",http://www.legalintelligence.com/documents/496...,https://www.navigator.nl/#/document/da5a4b72b4...,http://www.legalintelligence.com/documents/496...,None,None
550,ECLI:NL:RVS:1999:AA3693,1999-10-21,1999-01-01,None,1999-10-22,2010-07-22,H01.99.0579,H01990579,"\n \n <parablock xmlns=""http://www...",None,De staatssecretari

In [19]:
df_rs_cases.describe()

,ecli,uitspraakdatum,domain,instance,case_number,inhoudsindicatie,language,jurisdiction_country,jurisdiction_city,uitspraak_info,procedure_type,lodge_date,link,legislation_title,title,doctype,source
count,2632420,2632420,2349707,2632420,2603903,2632420,2632420,2632420,1407815,137176,554530,0,2632420,113881,0,2632420,2632420
unique,2632420,28054,31,1223,2345163,415179,1,1,48,136987,42,0,2,10265,0,1,1
top,ECLI:NL:RBNHO:2019:7177,2011-06-01,{Civiel recht},Rechtbank 's-Gravenhage,Onbekend,,NL,NL,'s-Gravenhage,"\n <bridgehead xmlns=""http://www.rechtspraa...",Hoger beroep,NaN,,Algemene wet bestuursrecht\n,NaN,decision,Rechtspraak
freq,1,1633,554240,251772,30796,2152106,2632420,2632420,389671,17,197399,NaN,1983110,11074,NaN,2632420,2632420


In [29]:
del df_li_cases, df_li_cases_raw, df_rs_opinions_raw, rs_only, rsli_intersection, map_rs, map_li, df_country_raw, df_rs_cases, df_rs_cases_raw, ecli, i, li_case, li_only, map_case_related_decisions, new_ecli, courts_list

NameError: name 'df_rs_cases_raw' is not defined

In [ ]:
print('Number of Cases in\nRS only cases: {}\nLI only cases: {}\nLI & RS cases: {}\nCase opinion: {}\nLegislation citation: {}\nCase citation: {}\nRelated decisions: {}'\
      .format(len(df_rs_only_cases), len(df_li_only_cases), len(df_rsli_intersection_cases), len(df_rs_opinions), len(df_rs_legislation_citations), len(df_rs_case_citations), len(df_rs_related_cases)))



---
## Data to DynamoDB

In [2]:
dynamodb = boto3.resource('dynamodb')  #, endpoint_url="http://localhost:8000")  # remove 'endpoint_url' to use web service

#### Table and key schema definition:

In [21]:
caselaw_table = dynamodb.create_table(
    TableName='Caselaw',
    KeySchema=[
        {
            'AttributeName': 'id',
            'KeyType': 'HASH'  # Partition key
        },
        {
            'AttributeName': 'doctype',
            'KeyType': 'RANGE'  # Sort key
        },
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'id',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'doctype',
            'AttributeType': 'S'
        },
        {
            "AttributeName": "court",
            "AttributeType": "S"
        },
                {
            "AttributeName": "date",
            "AttributeType": "S"
        },
    ],
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'GSI-court',
            'KeySchema': [
                {
                    'AttributeName': 'doctype',
                    'KeyType': 'HASH'
                },
                                {
                    'AttributeName': 'court',
                    'KeyType': 'RANGE'
                },
            ],
            'Projection': {
                'ProjectionType': 'ALL',
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 20,
                'WriteCapacityUnits': 20
            },
        },
                {
            'IndexName': 'GSI-date',
            'KeySchema': [
                {
                    'AttributeName': 'doctype',
                    'KeyType': 'HASH'
                },
                                {
                    'AttributeName': 'date',
                    'KeyType': 'RANGE'
                },
            ],
            'Projection': {
                'ProjectionType': 'ALL',
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 20,
                'WriteCapacityUnits': 20
            },
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 20,
        'WriteCapacityUnits': 20
    }
)
print("Table status:", caselaw_table.table_status)


Table status: CREATING


#### Functions to load data into table:

In [3]:
caselaw_table = dynamodb.Table('Caselaw')

def batch_load_data(table, data, output=False):
    keys = [key['AttributeName'] for key in table.key_schema]
    counter = 0
    with table.batch_writer() as batch:
        #for item in data:
        for item in data.iterrows():
            item = item[1].to_dict()
            counter += 1
            if output:
                values = [item[key] for key in keys]
                print("Adding item:", dict(zip(keys, values)))
            #batch.put_item(Item=item, ConditionExpression='attribute_not_exists(ecli)')
            #batch.put_item(Item=item, ConditionExpression=Attr('ecli').not_exists())
            batch.put_item(Item=item)
    print(f'{counter} items added.')

def read_csv(filename, items):
    data = csv.DictReader(open(filename))
    for item in data:
        items.append(item)


In [47]:
del df_country, df_court, df_li_only_cases, df_rs_opinions, df_rs_related_cases, df_rsli_intersection_cases

In [51]:
batch_load_data(caselaw_table, df_rs_only_cases)

2631803 items added.


In [52]:
batch_load_data(caselaw_table, df_li_only_cases)

44 items added.


In [49]:
batch_load_data(caselaw_table, df_rsli_intersection_cases)

617 items added.


In [50]:
batch_load_data(caselaw_table, df_rs_opinions)

65692 items added.


In [15]:
batch_load_data(caselaw_table, df_rs_case_citations)

KeyboardInterrupt: 

In [16]:
batch_load_data(caselaw_table, df_rs_legislation_citations)

18947 items added.


In [10]:
batch_load_data(caselaw_table, df_rs_related_cases)

137134 items added.


In [24]:
batch_load_data(caselaw_table, df_court)

1231 items added.


In [23]:
batch_load_data(caselaw_table, df_country)

36 items added.


#### Example queries:

Uitspraak by ecli:

In [4]:
pprint(caselaw_table.get_item(Key={'id': 'ECLI:NL:RBHAA:2006:1006',
                                   'doctype': 'uitspraak'}))

{'Item': {'abstract': None,
          'case_number': 'AWB- 05_6230',
          'country': 'NL',
          'court': 'Rechtbank Haarlem',
          'date': '2006-10-30T00:00:00',
          'description': '',
          'doctype': 'uitspraak',
          'id': 'ECLI:NL:RBHAA:2006:1006',
          'language': 'NL',
          'link': '',
          'lodge_date': '1900-01-01T00:00:00',
          'name': None,
          'procedure_type': None,
          'subject': {'Belastingrecht', 'Bestuursrecht'},
          'venue': 'Haarlem'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '455',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Wed, 27 Jan 2021 14:29:32 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3335037513',
                                      'x-amzn-requestid'

Opinion by ecli:

In [8]:
pprint(caselaw_table.query(
    ProjectionExpression="id, subject",
    KeyConditionExpression=Key('id').eq('ECLI:NL:RBHAA:2006:1006') &
                           Key('doctype').eq('uitspraak'))  # [lc, cc, d]
)


{'Count': 1,
 'Items': [{'id': 'ECLI:NL:RBHAA:2006:1006',
            'subject': {'Bestuursrecht', 'Belastingrecht'}}],
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '129',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Thu, 24 Dec 2020 11:54:19 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2305071713',
                                      'x-amzn-requestid': 'K13IJO7A3UP30NV5KJVIGG5BA3VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'K13IJO7A3UP30NV5KJVIGG5BA3VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
 'ScannedCount': 1}


Case citations by ecli:

In [9]:
pprint(caselaw_table.query(
    KeyConditionExpression=Key('id').eq('ECLI:NL:GHARN:1958:38') &
                           Key('doctype').begins_with('c-citation'))  # [lc, cc, d]
)

{'Count': 21,
 'Items': [{'doctype': 'c-citation-ECLI:NL:DETARCO:1953:AY3105',
            'id': 'ECLI:NL:GHARN:1958:38',
            'target_ecli': 'ECLI:NL:DETARCO:1953:AY3105'},
           {'doctype': 'c-citation-ECLI:NL:DETARCO:1953:AY3110',
            'id': 'ECLI:NL:GHARN:1958:38',
            'target_ecli': 'ECLI:NL:DETARCO:1953:AY3110'},
           {'doctype': 'c-citation-ECLI:NL:DETARCO:1953:AY3113',
            'id': 'ECLI:NL:GHARN:1958:38',
            'target_ecli': 'ECLI:NL:DETARCO:1953:AY3113'},
           {'doctype': 'c-citation-ECLI:NL:DETARCO:1953:AY3114',
            'id': 'ECLI:NL:GHARN:1958:38',
            'target_ecli': 'ECLI:NL:DETARCO:1953:AY3114'},
           {'doctype': 'c-citation-ECLI:NL:DETARCO:1953:AY3311',
            'id': 'ECLI:NL:GHARN:1958:38',
            'target_ecli': 'ECLI:NL:DETARCO:1953:AY3311'},
           {'doctype': 'c-citation-ECLI:NL:DETARCO:1953:AY3399',
            'id': 'ECLI:NL:GHARN:1958:38',
            'target_ecli': 'ECLI:NL:DETARCO

Cases per subject and court

In [15]:
pprint(caselaw_table.query(
    IndexName='GSI-court',  # only needs to be specified if secondary index is used
    ProjectionExpression="id, court, country",
    FilterExpression='contains(#cl, :val)',
    ExpressionAttributeNames={'#cl': 'subject'},
    ExpressionAttributeValues={':val': 'Belastingrecht'},
    KeyConditionExpression=Key('doctype').eq('uitspraak') & Key('court').begins_with('R'))
)

{'Count': 0,
 'Items': [],
 'LastEvaluatedKey': {'court': 'Raad van Beroep in Ambtenarenzaken van Aruba, '
                               'Curaçao, Sint Maarten en van Bonaire, Sint '
                               'Eustatius en Saba',
                      'doctype': 'uitspraak',
                      'id': 'ECLI:NL:ORBAACM:2017:5'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '251',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Thu, 24 Dec 2020 11:59:45 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '1437545301',
                                      'x-amzn-requestid': 'GJ9TKG17FC84V0G2KDA680EPUJVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'GJ9TKG17FC84V0G2KDA680EPUJVV4KQNSO5AEMVJF66Q9ASUAA

In [ ]:
''' usage example:

pprint(table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={"#yr": "year"},
    KeyConditionExpression=
            Key('year').eq(year) & Key('title').between(title_range[0], title_range[1]))
)
'''